## Load Packages

In [2]:
using JuMP;
using HiGHS;


Every morning, the newsboy goes to the editor of the newspaper and buys a quantity of $x$ of newspapers at a cost of $c$ per unit. This $x$ amount is limited upwards by a $u$ value, as the newsboy has finite purchasing power. The decision is made under uncertainty since the demand  $d$  newspapers of the day is unknown. The number of $y$ newspapers is sold at a price of $q$ per unit. The newsboy also has an agreement with the newspaper editor: the number of unsold newspapers $w$ can be returned to the editor, who pays a $r$ price for him.

Consider the two-stage formulation:

$$\min_{0 \leq x \leq u} cx + \mathbb{E}_d[Q(x,d)]$$

where, 

$$Q(x,d) = \min_{y, z} \left\{ -qy -rz | y \leq d; y + z \leq x; y, z \geq 0 \right\}$$ 

Let $c=10$, $r=5$, $q=25$ e $u=150$. Also, assume that the demand follows a (continuous) uniform probability distribution over the range $[50,150]$.

## 1) Find the optimal solution.

For this item, we will use the bilevel formulation of the newsboy problem:

$$
\begin{aligned}
    &\min && cx + \mathbb{E}_d[Q(x,d)] \\
    & st && 0 \leq x \leq u \\
        &&& Q(x,d) =
     \begin{aligned}[t]
        &\min && -qy-rz \\
            & st && y \leq d \\
            &&& y+z \leq x\\
            &&& y, z \geq 0\\
     \end{aligned}\\
\end{aligned}
$$

We then begin by solving the second stage problem. The solution is trivial, he must sell as many newspapers as possible and return the rest, that is:

$$y^* = min(d,x)$$
$$z^* = max(x-d,0)$$

So:

$$\mathbb{E}_d[Q(x,d)] = \mathbb{E}[-q min(d,x) + -r max(x-d,0)]$$

Using $f(t)$ like the probability distribution funcion of $d$, by the definition of expectation:

$$\mathbb{E}_d[Q(x,d)] = -\int_{-\infty}^x (qt + r(x-t))f(t)dt - \int^{\infty}_x qtf(t)dt$$ 

Using $F(t)$ like the cumulative probability distribution function of $d$, by the relationship between $f(t)$ and $F(t)$:

$$\mathbb{E}_d[Q(x,d)] = -(q-r)\int_{-\infty}^x tf(t)dt - rxF(x) - qx(1-F(x))$$

Integrating by parts:

$$\mathbb{E}_d[Q(x,d)] =  -qx + (q-r)\int_{-\infty}^x (t)dt $$

Differentiating:

$$(\mathbb{E}_d[Q(x,d)])' = -q + (q-r)F(x) $$

Then the derivative of the total cost:

$$(\mathbb{E}_d[cx+Q(x,d)])' = c-q + (q-r)F(x) $$

The optimal value (ignoring the bounds of x) must respect:

$$(c-q) + (q-r)F(x^*) = 0 $$

Finally, the solution:

$$\frac{q-c}{q-r} < F(0) \Rightarrow x^* = 0 $$
$$\frac{q-c}{q-r} > F(u) \Rightarrow x^* = u$$ 
$$F(0) \leq \frac{q-c}{q-r} \leq F(u) \Rightarrow x^* = F^{-1}(\frac{q-c}{q-r})$$

Using the data from the assignment:
$$\frac{q-c}{q-r} = \frac{25-10}{25-5} = 0.75 $$
$$x^* = F^{-1}(0.75) = 125$$

## 2) Formulate the first and second stage problem. Additionally, considering the probability distribution defined above, formulate and solve the deterministic equivalent problem.

$$
\begin{aligned}
    &\min && cx + \Sigma_s p_s(-qy_s-rz_s)  \\
    & st && 0 \leq x \leq u \\
        &&& y_s \leq d_s \\
        &&& y_s+z_s \leq x\\
        &&& y_s, z_s \geq 0\\
\end{aligned}
$$

In [31]:
struct NewsboyData
    u::Float64
    c::Float64
    q::Float64
    r::Float64
    d::Vector{Float64}
    p::Vector{Float64}
end

function get_newsboy_data(data::NewsboyData)
    u = data.u
    c = data.c
    q = data.q
    r = data.r
    d = data.d
    p = data.p
    return u, c, q, r, d, p
end

function newsboy_optimal(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    S = length(d)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x<=u)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])

    @constraint(model,[s = 1:S], y[s] + z[s] <= x)

    @objective(model, Min,c*x - sum(q*y[s]*p[s]+r*z[s]*p[s] for s = 1:S))

    optimize!(model)
    return objective_value(model), value(model[:x])
end

function newsboy_crystql_ball(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    S = length(d)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x[s=1:S]<=u)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])

    @constraint(model,[s = 1:S], y[s] + z[s] <= x[s])

    @objective(model, Min, sum(c*x[s] - q*y[s]*p[s]-r*z[s]*p[s] for s = 1:S))

    optimize!(model)
    return objective_value(model)
end

function newsboy_average(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    d = sum(d.*p)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x<=u)
    @variable(model, 0<=y<=d)
    @variable(model, 0<=z)

    @constraint(model, y + z <= x)

    @objective(model, Min, c*x - q*y - r*z )

    optimize!(model)
    return objective_value(model), value(model[:x])
end

newsboy_average (generic function with 1 method)

In [32]:
data = NewsboyData(150,10,25,5,LinRange(60,150,91),ones(91)./91)
model = newsboy(data)
objective_value(model)
value(model[:x])

128.0